In [2]:
import os
# from utils.Preprocessing import filter,features,plots
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
import datetime
warnings.filterwarnings("ignore")

This code uploads all files from a country

In [4]:
file_path = "Organized Data/"
file_list = os.listdir(file_path)
print(file_list)
df = pd.DataFrame()

for file in file_list:
    # if 'abb_china' in file and file.endswith(".csv"):
    print(file)
    df_temp = pd.read_csv(os.path.join(file_path, file), index_col=0)
    df = pd.concat([df, df_temp])

# dataset=dataset.sample(frac=1).reset_index(drop=True)
df.fillna(0, inplace=True)
df

['brazil', 'canada', 'china', 'france', 'germany', 'italy', 'singapore', 'switzerland', 'usa']
brazil


PermissionError: [Errno 13] Permission denied: 'Organized Data/brazil'

This code is used to upload data from a single file

In [27]:
# df=pd.read_csv('Organized Data/france/loreal_france_119961_statistics.csv')
# # Fill NaN values with zero
# df.fillna(0, inplace=True)
# df

,Unnamed: 0,TFF,CT,TAV,SHOT_COUNT,STATISTICS_ID,MOLD_ID,MOLD_CODE,COUNTER_ID,CDATA_ID,SC,WACT,ACT,TOOLING_TYPE,TOTAL_CAVITIES,max_shots,DATE,HOUR,LIFE_CYCLE,PHASE
0,1,20210429072248,484.0,229.0,4,103871,119961,TOOL003479,NCM2115I01032,103870,4,303.786088,280,injection,16,1000000,20210429,2021042907,0.000004,1
1,2,20210429082250,0.0,233.0,0,104816,119961,TOOL003479,NCM2115I01032,104815,4,303.786088,280,injection,16,1000000,20210429,2021042908,0.000004,1
2,3,20210429092250,0.0,233.0,0,104818,119961,TOOL003479,NCM2115I01032,104817,4,303.786088,280,injection,16,1000000,20210429,2021042909,0.000004,1
3,4,20210429102251,0.0,232.0,0,104820,119961,TOOL003479,NCM2115I01032,104819,4,303.786088,280,injection,16,1000000,20210429,2021042910,0.000004,1
4,5,20210429113017,0.0,232.0,0,105519,119961,TOOL003479,NCM2115I01032,105518,4,303.786088,280,injection,16,1000000,20210429,2021042911,0.000004,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18452,18461,20231101171922,303.0,433.0,39,10278399,119961,TOOL003479,EMA2303M10124,10278398,267490,303.786088,280,injection,16,1000000,20231101,2023110117,0.267490,2
18453,18462,20231101175948,303.0,430.0,80,10278582,119961,TOOL003479,EMA2303M10124,10278581,267609,303.786088,280,injection,16,1000000,20231101,2023110117,0.267609,2
18454,18463,20231101181931,303.0,435.0,39,10278585,119961,TOOL003479,EMA2303M10124,10278584,267609,303.786088,280,injection,16,1000000,20231101,2023110118,0.267609,2
18455,18464,20231101185958,303.0,430.0,80,10278774,119961,TOOL003479,EMA2303M10124,10278773,267727,303.786088,280,injection,16,1000000,20231101,2023110118,0.267727,2


In [44]:
df=df[['HOUR','CT', 'TAV', 'SHOT_COUNT','PHASE']]
# Assuming df is your DataFrame and 'HOUR' is your time column
df['HOUR'] = pd.to_datetime(df['HOUR'], format='%Y%m%d%H')
df = df.sort_values('HOUR')
df

,HOUR,CT,TAV,SHOT_COUNT,PHASE
Unnamed: 0,,,,,
161,2020-09-22 14:00:00,0.0,260,1,1
0,2020-09-22 14:00:00,0.0,284,6599,1
0,2020-09-22 14:00:00,0.0,265,40,1
0,2020-09-22 14:00:00,0.0,248,1,1
0,2020-09-22 14:00:00,0.0,270,7,1
...,...,...,...,...,...
18319,2023-10-31 12:00:00,0.0,221,0,4
16190,2023-10-31 12:00:00,0.0,209,0,4
18320,2023-10-31 13:00:00,0.0,222,0,4


In [45]:
# Selecting features
features_considered = ['CT', 'TAV', 'SHOT_COUNT']

# One-hot encoding the 'PHASE' feature
y = pd.get_dummies(df['PHASE']).values

# Creating the sliding window
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y[i:i + time_steps])
    return np.array(Xs), np.array(ys)


time_steps = 720  # Represents 30 days of hourly data

# reshape to [samples, time_steps, n_features]
X, y = create_dataset(df[features_considered], y, time_steps)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Testing set shape: {X_test.shape}, {y_test.shape}")

Training set shape: (159119, 720, 3), (159119, 720, 4)
Testing set shape: (39780, 720, 3), (39780, 720, 4)


In [46]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.layers import TimeDistributed

# Assuming X_train, y_train, X_test, y_test are your training and testing sets
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[2]

# Function to create the model
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(TimeDistributed(Dense(n_outputs, activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
# # Function to fine-tune hyperparameters
# def fine_tune_model(X_train, y_train):
#     model = KerasClassifier(build_fn=create_model, verbose=0)
#     # define the grid search parameters
#     optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
#     param_grid = dict(optimizer=optimizer)
#     grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
#     grid_result = grid.fit(X_train, y_train)
#     # summarize results
#     print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#     means = grid_result.cv_results_['mean_test_score']
#     stds = grid_result.cv_results_['std_test_score']
#     params = grid_result.cv_results_['params']
#     for mean, stdev, param in zip(means, stds, params):
#         print("%f (%f) with: %r" % (mean, stdev, param))
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Create the LSTM model
model = create_model()

# Train the model
model.fit(X_train, y_train, epochs=35, batch_size=12, verbose=2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=2,callbacks=[tensorboard_callback])

print(f"Loss: {loss}, Accuracy: {accuracy}")

# Fine-tune hyperparameters
# fine_tune_model(X_train, y_train)

Epoch 1/35
13260/13260 - 11334s - loss: 1.0874 - accuracy: 0.4969
Epoch 2/35
13260/13260 - 3032s - loss: 0.9342 - accuracy: 0.5681
Epoch 3/35
13260/13260 - 2200s - loss: 0.8665 - accuracy: 0.6024
Epoch 4/35
13260/13260 - 3109s - loss: 0.8166 - accuracy: 0.6247
Epoch 5/35
13260/13260 - 2743s - loss: 0.7819 - accuracy: 0.6395
Epoch 6/35
13260/13260 - 3198s - loss: 0.7648 - accuracy: 0.6478
Epoch 7/35
13260/13260 - 2137s - loss: 0.7615 - accuracy: 0.6507
Epoch 8/35


In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense, Bidirectional, LSTM

# # Define the BiLSTM model
# model = Sequential()
# model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(time_steps, n_features)))
# model.add(Dense(4, activation='softmax')) 

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# history = model.fit(X, y, epochs=10, verbose=1)

# # Evaluate the model
# loss, accuracy = model.evaluate(X, y)
# print('Loss: %.2f' % (loss*100))
# print('Accuracy: %.2f' % (accuracy*100))


In [32]:
model.save('Weights/france_119961.h5')

In [33]:
from keras.models import load_model
model=load_model('Weights/abb_china_36435.h5')

Testing on other dataset

In [34]:
df = pd.read_csv('Organized Data/france/loreal_france_119970_statistics.csv')
df.fillna(0, inplace=True)
df=df[['HOUR','CT', 'TAV', 'SHOT_COUNT','PHASE']]
# Assuming df is your DataFrame and 'HOUR' is your time column
df['HOUR'] = pd.to_datetime(df['HOUR'], format='%Y%m%d%H')
df = df.sort_values('HOUR')
features_considered = ['CT', 'TAV', 'SHOT_COUNT']

# One-hot encoding the 'PHASE' feature
y = pd.get_dummies(df['PHASE']).values

# Creating the sliding window
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y[i:i + time_steps])
    return np.array(Xs), np.array(ys)


time_steps = 720  # Represents 30 days of hourly data

# reshape to [samples, time_steps, n_features]
X, y = create_dataset(df[features_considered], y, time_steps)

loss, accuracy = model.evaluate(X, y, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy*100}")

497/497 - 34s - loss: 4.4136 - accuracy: 0.4681
Loss: 4.413638114929199, Accuracy: 46.807268261909485


Test for all files in a folder

In [35]:
# Folder containing CSV files
folder_path = 'Organized Data/france/'

# Loop through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        # Construct the full path to the CSV file
        file_path = os.path.join(folder_path, filename)

        # Read the CSV file
        df = pd.read_csv(file_path)
        df.fillna(0, inplace=True)
        df = df[['HOUR', 'CT', 'TAV', 'SHOT_COUNT', 'PHASE']]

        # Assuming df is your DataFrame and 'HOUR' is your time column
        df['HOUR'] = pd.to_datetime(df['HOUR'], format='%Y%m%d%H')
        df = df.sort_values('HOUR')
        features_considered = ['CT', 'TAV', 'SHOT_COUNT']

        # One-hot encoding the 'PHASE' feature
        y = pd.get_dummies(df['PHASE']).values

        # Creating the sliding window
        def create_dataset(X, y, time_steps=1):
            Xs, ys = [], []
            for i in range(len(X) - time_steps):
                v = X.iloc[i:(i + time_steps)].values
                Xs.append(v)
                ys.append(y[i:i + time_steps])
            return np.array(Xs), np.array(ys)

        time_steps = 720  # Represents 30 days of hourly data

        # Reshape to [samples, time_steps, n_features]
        X, y = create_dataset(df[features_considered], y, time_steps)

        # Evaluate the model on the data
        loss, accuracy = model.evaluate(X, y, verbose=2)

        # Print filename, loss, and accuracy
        print(f"File: {filename}, Loss: {loss}, Accuracy: {np.round(accuracy * 100,2)}")

555/555 - 45s - loss: 0.0653 - accuracy: 0.9735
File: loreal_france_119961_statistics.csv, Loss: 0.06526671350002289, Accuracy: 97.35
497/497 - 49s - loss: 4.4136 - accuracy: 0.4681
File: loreal_france_119970_statistics.csv, Loss: 4.413638114929199, Accuracy: 46.81
338/338 - 42s - loss: 7.3048 - accuracy: 0.3072
File: loreal_france_119990_statistics.csv, Loss: 7.304849624633789, Accuracy: 30.72
476/476 - 51s - loss: 3.3363 - accuracy: 0.4761
File: loreal_france_119994_statistics.csv, Loss: 3.3363375663757324, Accuracy: 47.61
588/588 - 68s - loss: 2.8121 - accuracy: 0.2842
File: loreal_france_119998_statistics.csv, Loss: 2.8121225833892822, Accuracy: 28.42
288/288 - 28s - loss: 6.2205 - accuracy: 0.2200
File: loreal_france_120006_statistics.csv, Loss: 6.220480918884277, Accuracy: 22.0
626/626 - 76s - loss: 2.0553 - accuracy: 0.5204
File: loreal_france_120010_statistics.csv, Loss: 2.0552713871002197, Accuracy: 52.04
521/521 - 57s - loss: 3.6314 - accuracy: 0.2693
File: loreal_france_1200

In [17]:
df=pd.read_csv('abb_germany_139229_statistics.csv')
df

,Unnamed: 0,TFF,CT,TAV,SHOT_COUNT,STATISTICS_ID,MOLD_ID,MOLD_CODE,COUNTER_ID,CDATA_ID,SC,WACT,ACT,TOOLING_TYPE,TOTAL_CAVITIES,max_shots
0,0,2.020053e+13,0.0,231.0,1,139523,139229,GHW1782996R08,NCM2021I01001,139517,1,139.141836,150.0,NaN,4,8000000
1,1,2.020053e+13,0.0,228.0,0,139531,139229,GHW1782996R08,NCM2021I01001,139525,1,139.141836,150.0,NaN,4,8000000
2,2,2.020053e+13,0.0,227.0,0,139539,139229,GHW1782996R08,NCM2021I01001,139533,1,139.141836,150.0,NaN,4,8000000
3,3,2.020053e+13,0.0,222.0,0,139547,139229,GHW1782996R08,NCM2021I01001,139541,1,139.141836,150.0,NaN,4,8000000
4,4,2.020053e+13,0.0,219.0,0,139555,139229,GHW1782996R08,NCM2021I01001,139549,1,139.141836,150.0,NaN,4,8000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6188,6188,2.023103e+13,140.0,250.0,61,1305827,139229,GHW1782999R0007,EMA2233M10401,1305826,559609,139.141836,150.0,NaN,4,8000000
6189,6189,2.023103e+13,140.0,250.0,197,1305861,139229,GHW1782999R0007,EMA2233M10401,1305860,559867,139.141836,150.0,NaN,4,8000000
6190,6190,2.023103e+13,140.0,250.0,61,1305864,139229,GHW1782999R0007,EMA2233M10401,1305863,559867,139.141836,150.0,NaN,4,8000000
6191,6191,2.023103e+13,140.0,265.0,197,1305899,139229,GHW1782999R0007,EMA2233M10401,1305898,560120,139.141836,150.0,NaN,4,8000000


In [18]:
df['HOUR'] = df['TFF'][:-4]
df['HOUR'] = pd.to_datetime(df['HOUR'],format = '%Y%m%d%H%M%S')
df.sort_values(by = 'HOUR', ascending = True,inplace=True)
df

,Unnamed: 0,TFF,CT,TAV,SHOT_COUNT,STATISTICS_ID,MOLD_ID,MOLD_CODE,COUNTER_ID,CDATA_ID,SC,WACT,ACT,TOOLING_TYPE,TOTAL_CAVITIES,max_shots,HOUR
23,23,1.970010e+13,466.0,255.0,1,140029,139229,GHW1782996R08,NCM2021I01001,140023,14,139.141836,150.0,NaN,4,8000000,1970-01-01 02:00:00
35,35,1.970010e+13,54.0,269.0,3,140174,139229,GHW1782996R08,NCM2021I01001,140168,43,139.141836,150.0,NaN,4,8000000,1970-01-01 02:00:00
34,34,1.970010e+13,0.0,263.0,0,140164,139229,GHW1782996R08,NCM2021I01001,140158,40,139.141836,150.0,NaN,4,8000000,1970-01-01 02:00:00
33,33,1.970010e+13,0.0,261.0,0,140156,139229,GHW1782996R08,NCM2021I01001,140150,40,139.141836,150.0,NaN,4,8000000,1970-01-01 02:00:00
32,32,1.970010e+13,0.0,261.0,0,140148,139229,GHW1782996R08,NCM2021I01001,140142,40,139.141836,150.0,NaN,4,8000000,1970-01-01 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2631,2631,NaN,0.0,NaN,0,1042250,139229,GHW1782999R0007,EMA2233M10401,1042249,116652,139.141836,150.0,NaN,4,8000000,NaT
6189,6189,2.023103e+13,140.0,250.0,197,1305861,139229,GHW1782999R0007,EMA2233M10401,1305860,559867,139.141836,150.0,NaN,4,8000000,NaT
6190,6190,2.023103e+13,140.0,250.0,61,1305864,139229,GHW1782999R0007,EMA2233M10401,1305863,559867,139.141836,150.0,NaN,4,8000000,NaT
6191,6191,2.023103e+13,140.0,265.0,197,1305899,139229,GHW1782999R0007,EMA2233M10401,1305898,560120,139.141836,150.0,NaN,4,8000000,NaT


In [1]:
from datetime import datetime

# Dates
start_date = datetime.strptime('4/14/2023 3:00:00 AM', '%m/%d/%Y %I:%M:%S %p')
end_date = datetime.strptime('10/7/2026 11:00:00 PM', '%m/%d/%Y %I:%M:%S %p')

# Total hours
total_hours = int((end_date - start_date).total_seconds() / 3600) + 1  # +1 to include the end hour

# Window size
window_size = 720

# Number of samples
num_samples = total_hours - window_size + 1

print(f'Total hours: {total_hours}')
print(f'Number of samples (batch size for using all data): {num_samples}')


Total hours: 30549
Number of samples (batch size for using all data): 29830
